In [1]:
import pandas as pd
from datetime import date, timedelta
import mysql.connector

In [170]:
"""
status:
    1 - active
    2 - cancelled
    3 - deactivated

sub_start
sub_end

period_start
period_end

    base:
        sub_start <= period_start                         -> if status = 1 or 2
        sub_start <= period_start <= sub_end              -> if status = 3
    
    new:
        period_start <= FIRST sub_start <= period_end
    
    churned:
        period_start <= sub_end <= period_end
    
    resurrected:
        if AT LEAST ONE but NOT THE FIRST sub_start is contained in [period_start, period_end]

A person can be present in more than one category

"""

'\nstatus:\n    1 - active\n    2 - cancelled\n    3 - deactivated\n\nsub_start\nsub_end\n\nperiod_start\nperiod_end\n\n    base:\n        sub_start <= period_start                         -> if status = 1 or 2\n        sub_start <= period_start <= sub_end              -> if status = 3\n    \n    new:\n        period_start <= FIRST sub_start <= period_end\n    \n    churned:\n        period_start <= sub_end <= period_end\n    \n    resurrected:\n        if AT LEAST ONE but NOT THE FIRST sub_start is contained in [period_start, period_end]\n\nA person can be present in more than one category\n\n'

### Function Definitions

In [10]:
# if our data was real time, in order to consider last p days starting from yesterday:
def set_period(p):
    yesterday = date.today() - timedelta(days = 2)
    period = timedelta(days = p-1)
    
    period_start = yesterday - period
    period_end = yesterday

    return(period_start, period_end)

In [27]:
# But our datebase is  frozen on oct 31st 2022, yesterday will not work
def set_period(p):
    period_end = date(2022,10,31)
    period_start = period_end - timedelta(days = p-1)

    return(period_start, period_end)

In [29]:
def is_base(df, period_start, period_end):
    for i in range(len(df)):
        if df['status'][i] == 3:
            if df['sub_start'][i].date() <= period_start <= df['sub_end'][i].date():
                return (True,i)
        else:
            if df['sub_start'][i].date() <= period_start:
                return (True,i)
    return (False, i)

In [31]:
def is_new(df, period_start,period_end):
    if period_start <= df['sub_start'].min().date() <= period_end:
        return (True,0)
        
    return (False, None)

In [5]:
def is_churned(df,period_start, period_end):
    for i in range(len(df)):
        if pd.isnull(df['sub_end'][i]):
            continue
        
        if period_start <= df['sub_end'][i].date() <= period_end:
            return (True,i)
            
    return (False, None)            

In [9]:
def is_resurrected(df, period_start,period_end):
    for i in range(1,len(df)):
        if period_start <= df['sub_start'][i].date() <= period_end:
            
            return (True,i)
        
    return (False, None)

In [23]:
def generate_user_tags(df, period = 30, period_start = None, period_end = None):
    if period_start == None and period_end == None:
        period_start, period_end = set_period(30)
        
    if  period_start == None:
        period_start = period_end - timedelta(days = period -1)

    if period_end == None:
        period_end = period_start + timedelta(days = period +1)
    r = []
    users = df['user_id'].unique()

    for user_id in users:
        mask = df['user_id'] == user_id
        user = df[mask].reset_index(drop = True)      

        result, index = is_base(user, period_start, period_end)
        if test:
            r.append([user['user_id'][index], user['status'].iloc[-1],user['plan'][index],'Base'])
            
        result, index = is_new(user, period_start, period_end)
        if test:
            r.append([user['user_id'][index], user['status'].iloc[-1],user['plan'][index],'New'])
            
        result, index = is_churned(user, period_start, period_end)
        if test:
            r.append([user['user_id'][index], user['status'].iloc[-1],user['plan'][index],'Churned'])
            
        result, index = is_resurrected(user, period_start, period_end)
        if test:
            r.append([user['user_id'][index], user['status'].iloc[-1],user['plan'][index],'Resurrected'])

    result = pd.DataFrame(r, columns = ["user_id", "current status", "plan", "tag"])
    result.sort_values(by = 'user_id', inplace = True, ignore_index = True)

    return result

### Data exploration

In [13]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="admin",
    database="growth_data",use_pure = True
)
query = """
SELECT
	user_id,
    subscription_id,
    CASE
		WHEN subscription_type = 0 THEN "Monthly"
        WHEN subscription_type = 1 THEN "Quarterly"
        WHEN subscription_type = 2 THEN "Annual"
	END as plan,
    subscription_status as status,
    CAST(date_start as DATE) as sub_start,
    CAST(date_deactivated as DATE) as sub_end
    FROM
	student_subscriptions
WHERE
	subscription_type != 3
ORDER BY user_id, date_start;
"""

# Load into DataFrame
df = pd.read_sql(query, con=conn)

# Preview
print(df.head())

conn.close()

C:\Users\User\AppData\Local\Temp\ipykernel_12732\2039296800.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=conn)


   user_id  subscription_id     plan  status   sub_start     sub_end
0       31             8368   Annual       1  2022-03-31        None
1      182             8246  Monthly       3  2022-03-26  2022-06-08
2      182             9415  Monthly       3  2022-07-13  2022-08-13
3      221             4629  Monthly       3  2021-06-10  2021-07-10
4      235             6429   Annual       1  2021-11-28        None


In [14]:
df.head()

,user_id,subscription_id,plan,status,sub_start,sub_end
0,31,8368,Annual,1,2022-03-31,None
1,182,8246,Monthly,3,2022-03-26,2022-06-08
2,182,9415,Monthly,3,2022-07-13,2022-08-13
3,221,4629,Monthly,3,2021-06-10,2021-07-10
4,235,6429,Annual,1,2021-11-28,None


In [29]:
df.status.unique()
# 1- active, 2 - cancelled, 3 - deactivated

array([1, 3, 2], dtype=int64)

Cancelled means use canceled before the subscription period ends, so still has access to platform. When the current billing period end(sub_end column), status of that user switches to deactivated

In [17]:
df['sub_start'] = pd .to_datetime(df['sub_start'], infer_datetime_format = True)
df['sub_end'] = pd .to_datetime(df['sub_end'], infer_datetime_format = True)

C:\Users\User\AppData\Local\Temp\ipykernel_12732\2164813180.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['sub_start'] = pd .to_datetime(df['sub_start'], infer_datetime_format = True)
C:\Users\User\AppData\Local\Temp\ipykernel_12732\2164813180.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['sub_end'] = pd .to_datetime(df['sub_end'], infer_datetime_format = True)


In [34]:
df.head()

,user_id,subscription_id,plan,status,sub_start,sub_end
0,31,8368,Annual,1,2022-03-31,NaT
1,182,8246,Monthly,3,2022-03-26,2022-06-08
2,182,9415,Monthly,3,2022-07-13,2022-08-13
3,221,4629,Monthly,3,2021-06-10,2021-07-10
4,235,6429,Annual,1,2021-11-28,NaT


### Testing

In [33]:
df_test = generate_user_tags(df,period = 365)
df_test.head()

NameError: name 'test' is not defined

In [180]:
# let's choose one user_id and see whether our functions work correctly
df.loc[df['user_id'] == 182]

,user_id,subscription_id,plan,status,sub_start,sub_end
1,182,8246,Monthly,3,2022-03-26,2022-06-08
2,182,9415,Monthly,3,2022-07-13,2022-08-13


In [182]:
df_test.loc[df_test['user_id'] == 182]
# it works!

,user_id,current status,plan,tag
1,182,3,Monthly,New
2,182,3,Monthly,Resurrected
3,182,3,Monthly,Churned


In [188]:
result_last_30 = generate_user_tags(df, period = 30)
result_last_90 = generate_user_tags(df, period = 90)
result_last_365 = generate_user_tags(df, period = 365)